In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install sacremoses

In [ ]:
!pip install flair==0.10
!pip install seqeval

In [ ]:
!pip install fasttext

In [ ]:
# !unrar x -Y "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.rar" "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/"

In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
ROOT = "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen"
MODEL = "BiLSTM-CRF-word2vec-sg."
EMBEDDINGS = "w2vec_wiki_id_case_300_sg"
FINAL_MODEL_NAME = "word2vec-sg"

In [ ]:
# !unrar x "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.rar" "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/"

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'kata', 1: 'label'}

# this is the folder in which train, test and dev files reside

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(f"{ROOT}/nested", columns,
                              train_file='flair-train.txt',
                              test_file='flair-test.txt',
                              dev_file='flair-test.txt')

2022-05-18 12:37:51,548 Reading data from /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested
2022-05-18 12:37:51,549 Train: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-train.txt
2022-05-18 12:37:51,555 Dev: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-test.txt
2022-05-18 12:37:51,556 Test: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-test.txt


In [ ]:
print(corpus.train[0].to_tagged_string('label'))

Dan jika orang yang berhutang itu dalam kesukaran maka berilah tangguh sampai dia berkelapangan Dan menyedekahkan sebagian atau semua utang itu lebih baik bagimu jika kamu mengetahui


In [ ]:
tag_type = 'label'

tag_dictionary = corpus.make_label_dictionary(tag_type)

2022-05-18 12:37:54,500 Computing label dictionary. Progress:


100%|██████████| 763/763 [00:00<00:00, 4253.02it/s]

2022-05-18 12:37:54,688 Corpus contains the labels: label (#29729)
2022-05-18 12:37:54,692 Created (for label 'label') Dictionary with 7 tags: <unk>, O, B-PER, I-PER, O+I-PER, O+I-PER+I-PER, O+I-PER+I-PER+I-PER


In [ ]:
print(tag_dictionary)

Dictionary with 7 tags: <unk>, O, B-PER, I-PER, O+I-PER, O+I-PER+I-PER, O+I-PER+I-PER+I-PER


In [ ]:
import gensim
import fasttext
import fasttext.util

In [ ]:
# import gensim

# word_vectors = gensim.models.Word2Vec.load('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_wiki_id_case_300_sg')
# word_vectors.wv.save_word2vec_format(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_wiki_id_case_300_sg.txt', binary=False)

# word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_wiki_id_case_300_sg.txt', binary=False)
# word_vectors.save('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_wiki_id_case_300_sg')

In [ ]:
from flair.embeddings import FastTextEmbeddings, WordEmbeddings

embedding = WordEmbeddings(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/{EMBEDDINGS}')

In [ ]:
from flair.data import Sentence
embedding.embed(Sentence('12'))

[Sentence: "12"   [− Tokens: 1]]

In [ ]:
# import fasttext.util
# ft = fasttext.load_model('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.bin')
# ft.get_word_vector('12')

In [ ]:
import requests, os
session = requests.Session()
session.verify = False
session.trust_env = False
os.environ['CURL_CA_BUNDLE']="" # or whaever other is interfering with 

from flair.models import SequenceTagger
from flair.embeddings import WordEmbeddings, FastTextEmbeddings
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embedding,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

SequenceTagger(
  (embeddings): WordEmbeddings(
    '/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_wiki_id_case_300_sg'
    (embedding): Embedding(481934, 300)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=9, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)


In [ ]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
history = trainer.train(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-{FINAL_MODEL_NAME}',
              learning_rate=0.1,
              max_epochs=300,
              )

2022-05-18 12:38:06,369 ----------------------------------------------------------------------------------------------------
2022-05-18 12:38:06,373 Model: "SequenceTagger(
  (embeddings): WordEmbeddings(
    '/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_wiki_id_case_300_sg'
    (embedding): Embedding(481934, 300)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=9, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2022-05-18 12:38:06,377 ----------------------------------------------------------------------------------------------------
2022-05-18 12:38:06,381 Corpus: "Corpus: 763 train + 763 dev + 763 test sentences"
2022-05-18 12:38:06,383 -------------------------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there is a model from previous trainings"


2022-05-18 12:38:07,107 epoch 1 - iter 2/24 - loss 1.33544746 - samples/sec: 132.93 - lr: 0.100000
2022-05-18 12:38:07,624 epoch 1 - iter 4/24 - loss 0.93922630 - samples/sec: 125.31 - lr: 0.100000
2022-05-18 12:38:08,320 epoch 1 - iter 6/24 - loss 0.74314663 - samples/sec: 92.63 - lr: 0.100000
2022-05-18 12:38:08,847 epoch 1 - iter 8/24 - loss 0.65121490 - samples/sec: 122.22 - lr: 0.100000
2022-05-18 12:38:09,670 epoch 1 - iter 10/24 - loss 0.57067159 - samples/sec: 78.05 - lr: 0.100000
2022-05-18 12:38:10,305 epoch 1 - iter 12/24 - loss 0.53616698 - samples/sec: 101.42 - lr: 0.100000
2022-05-18 12:38:10,785 epoch 1 - iter 14/24 - loss 0.50908568 - samples/sec: 134.76 - lr: 0.100000
2022-05-18 12:38:11,447 epoch 1 - iter 16/24 - loss 0.48984196 - samples/sec: 97.76 - lr: 0.100000
2022-05-18 12:38:12,265 epoch 1 - iter 18/24 - loss 0.47125459 - samples/sec: 79.02 - lr: 0.100000
2022-05-18 12:38:13,232 epoch 1 - iter 20/24 - loss 0.45132800 - samples/sec: 66.72 - lr: 0.100000
2022-05-1

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-{FINAL_MODEL_NAME}/best-model.pt')

2022-05-18 12:48:33,603 loading file /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-word2vec-sg/best-model.pt


In [ ]:
# create example sentence
sentence = Sentence('yaitu mereka orang-orang yang dimurkai allah')
# predict the tags
model.predict(sentence)

print(sentence.to_tagged_string())

yaitu mereka orang-orang <B-PER> yang <I-PER> dimurkai <I-PER> allah <I-PER>


In [ ]:
for entity in sentence.get_spans('label'):
    print(entity)

Span [3,4,5,6]: "orang-orang yang dimurkai allah"   [− Labels: PER (0.8365)]


In [ ]:
sentence.to_dict('label')

{'entities': [{'end_pos': 44,
   'labels': [PER (0.8365)],
   'start_pos': 13,
   'text': 'orang-orang yang dimurkai allah'}],
 'labels': [],
 'text': 'yaitu mereka orang-orang yang dimurkai allah'}

In [ ]:
from ast import literal_eval

test_data = pd.read_csv(f"{ROOT}/nested/test_data.csv")

test_data['sent'] = test_data['sent'].apply(literal_eval)
test_data['tag'] = test_data['tag'].apply(literal_eval)
test_data['pos'] = test_data['pos'].apply(literal_eval)

ii_lab = []
for i, d in test_data.iterrows() :
  sent = ' '.join(d.sent)

  sentence = Sentence(sent)

  model.predict(sentence)

  entities = sentence.get_labels('label')
  lab_idx = []
  entity_dict = {}
  for entity in entities:
      entity_in_str = str(entity)
      idx = re.findall(r"\d+", entity.identifier)
      idx = [int(i) for i in idx]
      idx = [i - 1 for i in idx]
      if "+I" in entity_in_str : 
        entity_dict[idx[0]] = entity_in_str.split()[0]

      lab_idx = lab_idx + idx

  new_lab_idx = []
  for i in range(len(d.sent)) : 
    if i not in lab_idx : 
      new_lab_idx.append(-999)

    else : 
      new_lab_idx.append(i)

  final_lab_idx = []
  i = 0
  for idx in new_lab_idx :
    if idx == -999 : 
      final_lab_idx.append('O')
      i = 0
    elif idx in entity_dict : 
      final_lab_idx.append(entity_dict[idx])
    else : 
      i += 1
      if i == 1 : 
        final_lab_idx.append('B-PER')
      else : 
        final_lab_idx.append('I-PER')

  ii_lab.append(final_lab_idx)


In [ ]:
test_data['pred'] = ii_lab
y_pred = ii_lab
y_true = test_data['tag'].to_list()

In [ ]:
def get_level_token(seqs, max_lev=2, level=0) : 
  new_all_level = []
  for seq in seqs : 
    new_all_level_i = []
    for tag in seq : 
      new_tag = []
      tag_per_level = tag.split('+')
      for lev in range(max_lev) :
        if lev + 1 > len(tag_per_level) : 
          new_tag.append(tag_per_level[-1])
        else : 
          new_tag.append(tag_per_level[lev])
      new_all_level_i.append(new_tag)
    new_all_level.append(new_all_level_i)
  
  new_seq = []
  for seq in new_all_level :
    seq_i = [] 
    for tag in seq : 
      seq_i.append(tag[level])
    new_seq.append(seq_i)
  
  return new_seq

y_true_0 = get_level_token(y_true, max_lev=2, level=0)
y_pred_0 = get_level_token(y_pred, max_lev=2, level=0)

In [ ]:
from seqeval.metrics import classification_report
print(classification_report(y_true_0, y_pred_0))

              precision    recall  f1-score   support

         PER       0.78      0.59      0.67       223

   micro avg       0.78      0.59      0.67       223
   macro avg       0.78      0.59      0.67       223
weighted avg       0.78      0.59      0.67       223



In [ ]:
y_true_1 = get_level_token(y_true, max_lev=2, level=1)
y_pred_1 = get_level_token(y_pred, max_lev=2, level=1)

from seqeval.metrics import classification_report
print(classification_report(y_true_1, y_pred_1))

              precision    recall  f1-score   support

         PER       0.71      0.53      0.61       223

   micro avg       0.71      0.53      0.61       223
   macro avg       0.71      0.53      0.61       223
weighted avg       0.71      0.53      0.61       223



In [ ]:
y_test_token = []
for i in y_true : 
  y_test_token = y_test_token + i

y_pred_token = []
for i in y_pred : 
  y_pred_token = y_pred_token + i

from sklearn.metrics import classification_report
print(classification_report(y_test_token, y_pred_token))


              precision    recall  f1-score   support

       B-PER       0.90      0.68      0.77       222
       I-PER       0.86      0.56      0.68       443
           O       0.95      0.99      0.97      6244
     O+I-PER       0.00      0.00      0.00       114

    accuracy                           0.94      7023
   macro avg       0.68      0.56      0.61      7023
weighted avg       0.92      0.94      0.93      7023



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
